In [26]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
# Télécharger les données de stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

from sklearn.feature_extraction.text import TfidfVectorizer



# Charger les DataFrames)
df = pd.read_csv("Data_Projet/Datafiniti_Hotel_Reviews.csv", sep=',', usecols=range(20))

# Remplacer les valeurs NaN dans la colonne "reviews.title" par une chaîne vide
df["reviews.title"] = df["reviews.title"].fillna("")

# Sélectionner les colonnes qui nous intéressent
df = df[["name", "reviews.rating", "reviews.text", "reviews.title"]]

# Concaténer les colonnes reviews.text et reviews.title
df["reviews_combined"] = df["reviews.title"] + " " + df["reviews.text"]

# Trier le DataFrame par la colonne "reviews.rating"
df = df.sort_values(by="reviews.rating")

# Supprimer les colonnes "reviews.text" et "reviews.title"
df.drop(columns=["reviews.text", "reviews.title"], inplace=True)

df["reviews_combined"] = df["reviews_combined"].fillna("")
stop_words = set(stopwords.words("english"))
# Initialiser le lemmatiseur
lemmatizer = WordNetLemmatizer()

# Fonction pour retirer les stopwords d'une chaîne de texte
def remove_stopwords(text):
    # Tokeniser le texte en mots
    tokens = word_tokenize(text)
    # Retirer les mots qui sont des stopwords
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    # Lemmatisation des mots
    lemmatized_tokens = [lemmatizer.lemmatize(word.lower()) for word in filtered_tokens]
    # Rejoindre les tokens lemmatisés en une chaîne de texte
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text

# Appliquer la fonction remove_stopwords à la colonne "reviews_combined"
df["reviews_combined_no_stopwords"] = df["reviews_combined"].apply(remove_stopwords)

tfidf_vectorizer = TfidfVectorizer(
    lowercase=True,  # Convertit le texte en minuscules
    tokenizer=word_tokenize,  # Tokenise les mots
    stop_words='english',  # Retire les mots vides en anglais
    max_df=0.8,  # Ignore les mots qui apparaissent dans plus de 80% des documents
    min_df=5  # Ignore les mots qui n'apparaissent pas au moins dans 5 documents
)

tfidf_matrix = tfidf_vectorizer.fit_transform(df["reviews_combined_no_stopwords"])


# Vérifier que les colonnes ont été supprimées avec succès
df



[nltk_data] Downloading package punkt to /Users/lucas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/lucas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/lucas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/Users/lucas/Documents/A5/4-VM IA/cours2/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<10000x4111 sparse matrix of type '<class 'numpy.float64'>'
	with 264526 stored elements in Compressed Sparse Row format>